
# Mapping Groundwater Contaminants of California

Potential goals of this notebook:
1. Clean the dataset into a workable dataframe
2. Spatially plot the data using geopandas, or cartopy
3. ...

## 1. Reorganizing the data

This first section will deal with cleaning and reorganizing the data. 

(The `%matplotlib inline` syntax prints out the figures that are created after each specific call.)

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

## Imports
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', 500)

## Set the data's directory path
script_dir = os.path.abspath('')
data_dir   = os.path.join( os.path.split(os.path.split(script_dir)[0])[0], # shared path
                           'whw2019_CA_data\\whw2019_NWQData' )

The data we are using for this analysis are from a collaboration between the United States Geological Survey ([USGS](https://www.usgs.gov/)), the Environmental Protection Agency([EPA](https://www.epa.gov/)), United States Department of Agriculture Agricultural Reaseach Service ([USDA ARS](https://www.ars.usda.gov/)), and the National Water Quality Monitoring Council ([NWQMC](https://acwi.gov/monitoring/)). The groundwater quality data was aggragated and downloaded from the [Water Quality Portal](https://www.waterqualitydata.us/coverage/). 

The reported data sources are:
* National Water Information System ([NWIS](https://waterdata.usgs.gov/nwis)) - USGS
* STOrage and RETrieval ([STORET](https://www.epa.gov/waterdata/water-quality-data-wqx)) Data Warehouse - EPA
* Sustaining The Earth's Watersheds - Agricultural Research Database System ([STEWARDS]())

For now the state/region of interest is California (CA). However, we hope to be able to apply similar analyses to other states around the US, or to other countries (e.g., India) should adequate spatial (X,Y,Z) and temporal data resolution be available.

To read in the datafiles, we must make the proper call toward their storage location (on Hydroshare). The following `pd.read_csv` commands may present with some warnings after running. In this instance, the warnings are fine to ignore (however, always be mindful of the coding issues). 


In [2]:
# Enter state code in 'state' variable to read in that states data results.
state = 'CA'
results  = pd.read_csv(r'{}\\{}_result.csv'.format(data_dir, state))
stations = pd.read_csv(r'{}\\{}_station.csv'.format(data_dir, state))

C:\Users\allan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (9,10,11,12,13,14,16,18,19,23,33,37,39,42,47,57,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\allan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
results.drop(columns=['OrganizationIdentifier', 
                      'OrganizationFormalName',
                      'ActivityIdentifier', 
                      'ActivityTypeCode', 
                      'ActivityMediaName',
                      'ActivityMediaSubdivisionName',
                      'ResultStatusIdentifier', 
                      'StatisticalBaseCode', 
                      'ResultValueTypeName',
                      'ResultWeightBasisText', 
                      'ResultTimeBasisText',
                      'ResultTemperatureBasisText',
                      'ResultParticleSizeBasisText',
                      'PrecisionValue', 
                      'ResultCommentText',
                      'USGSPCode',
                      'ResultDepthHeightMeasure/MeasureValue',
                      'ResultDepthHeightMeasure/MeasureUnitCode',
                      'ResultDepthAltitudeReferencePointText', 
                      'SubjectTaxonomicName',
                      'SampleTissueAnatomyName',
                      'ResultAnalyticalMethod/MethodIdentifier',
                      'ResultAnalyticalMethod/MethodIdentifierContext',
                      'ResultAnalyticalMethod/MethodName', 
                      'MethodDescriptionText',
                      'LaboratoryName',
                      'AnalysisStartDate', 
                      'ResultLaboratoryCommentText',
                      'DetectionQuantitationLimitTypeName',
                      'DetectionQuantitationLimitMeasure/MeasureValue',
                      'DetectionQuantitationLimitMeasure/MeasureUnitCode',
                      'PreparationStartDate', 
                      'ProviderName',
                      'ProjectIdentifier',
                      'ActivityConductingOrganizationText',
                      'ActivityCommentText',
                      'MeasureQualifierCode', 
                      'SampleCollectionMethod/MethodIdentifier',
                      'SampleCollectionMethod/MethodIdentifierContext',
                      'SampleCollectionMethod/MethodName',
                      'SampleCollectionEquipmentName',
                      'ResultDetectionConditionText'
                     ], inplace=True)

# # preview the data
# results.head()
# stations.head()

After uploading/reading the .csv file as a `Pands` dataframe, we dropped unnecessary column values (above). The next step is to merge the two data frames `results` and `stations` by the station identifier to obtain a unified dataframe.

We have reset the "headers" of the rows to be the 'MonitoringLocationIdentifier' for each station. The new variable `mwd` stands for "merged well dataframe".

The bounding box for Califonia was obtained [here](https://gist.github.com/jakebathman/719e8416191ba14bb6e700fc2d5fccc5).

In [4]:
# california bounding box information
ca_max_lon = -114.1315
ca_min_lon = -124.6509
ca_max_lat = 42.0126
ca_min_lat = 32.5121

# creat boolean conditional statements
lon_condition = (ca_min_lon <= stations['LongitudeMeasure']) & (stations['LongitudeMeasure'] <= ca_max_lon)
lat_condition = (ca_min_lat <= stations['LatitudeMeasure']) & (stations['LatitudeMeasure'] <= ca_max_lat)

In [ ]:
# Check for removal of inaccurate points
# stations.shape

# clean the data to drop stations outside the bounding box of California
stations = stations.loc[ (lat_condition & lon_condition) ,:]

# Check for removal of inaccurate points
# stations.shape

In [ ]:
mwd = stations.merge( results, on='MonitoringLocationIdentifier' )
# mwd = mwd.set_index( 'MonitoringLocationIdentifier' )

Now that we have a `Pandas` dataframe, I want to convert the dataframe to a geopandas dataframe.

If this is your first time using `GeoPandas`, you may need to install `GeoPandas`. The simplest way to do this is to use your `anaconda prompt` to call on a miniconda installation to download `GeoPandas` from `conda-forge`.

Essentially, enter the following into your `anaconda prompt` command shell:
`conda install -c conda-forge geopandas`

The installation make take a minute. Be patient - go grab coffee or something.

In [ ]:
# import geopandas
import geopandas as gpd
from shapely.geometry import Point

# Convert the DataFrame's Lat/Long coordinates into the appropriate shapely geometries
gpdgeom = [Point(xy) for xy in zip(mwd.LongitudeMeasure, mwd.LatitudeMeasure)]

# convert the dataframe to a geopandas dataframe
crs = {'init': 'epsg:4326'} # coordinate reference system -> 4326 - web mercator projection
# could also choose to be in UTM for CA?
mwgd = gpd.GeoDataFrame(mwd, crs=crs, geometry=gpdgeom)

Now that we have a `GeoPandas` geodataframe, we can begin to spatially plot the data over a map of California (or our area of interest).

However, the first step in the process will be to subset the data before plotting.

* Date - 'ActivityStartDate'
* Sample value - 'ResultMeasureValue'

Look into 'tidy data'.

In [ ]:
from datetime import datetime

# create a column of datetime variables
mwgd['ActivityStartDate'] = pd.to_datetime( mwgd['ActivityStartDate'] )


## Need to add a functionality that converts concentrations to proper units

When using chemicals/contaminants/characteristics other than Arsenic.


In [ ]:
# add row of values scaling As by the MCL
def determine_MCL( chemical ):
    if chemical == 'Arsenic':
        MCL_value = 0.010
    elif chemical == 'Nitrate':
        MCL_value = 45
    elif chemical == 'Chromium':
        MCL_value = 0.05
    elif chemical == 'Uranium':
        MCL_value = 0.030
    elif chemical == 'Selenium':
        MCL_value = 0.050
    elif chemical == 'Flouride':
        MCL_value = 2.0
    elif chemical == 'Total Dissolved Solids':
        MCL_value = 500
    elif chemical == 'Manganese':
        MCL_value = 0.05
    elif chemical == 'Iron':
        MCL_value = 0.3
    elif chemical == '0.15':
        MCL_value = 0.015
    return MCL_value

# above function returns the given MCL for the contaminant in mg/L

# ***** # building function to convert concentration data
# def convert_concentration( value_zip ):
#     for value, units in value_zip:
#         if 'mg' not in units:
#             if 'ug' in units: # micro-grams
#                 value = value/1e3
#     return value


## Figure Goal 3. 

Make a figure with two rows of subplots to show the temporal variability of concentrations in the Central Valley Aquifer System throughout the CA drought form 2011-2017. The figures will depict 2010-2018.

We will build a function to automate the process of subsetting the data.


In [ ]:
# this a function to autmoate the data subsetting process
def subset_data(dataset,
                year_of_int=2015,
                aquifer_of_int='Central Valley aquifer system', 
                chem_of_int='Arsenic'):
    
    # obtain all the samples during the year specified
    annual_cond = ( (dataset['ActivityStartDate'] > datetime(year_of_int-1,12,31)) &
                    (dataset['ActivityStartDate'] < datetime(year_of_int+1, 1, 1)) )
    annual_subset = mwgd.loc[ annual_cond, : ]
    
    # select a particular aquifer
    aqui_ann_subset = annual_subset.loc[ annual_subset['AquiferName'] == aquifer_of_int, :]

    # obtain all chemical rows
    MCL_value = determine_MCL( chem_of_int )
    chem_cond = aqui_ann_subset['CharacteristicName']  == chem_of_int
    chem_aqui_ann_subset = aqui_ann_subset.loc[ (chem_cond) ,:]

    # convert Result measure value
    if chem_of_int == 'Arsenic':
        # convert units to proper mg/L      
        chem_aqui_ann_subset['ConvertedResultMeasureValue'] = \
                chem_aqui_ann_subset['ResultMeasureValue']/1e3
        chem_aqui_ann_subset['NormalizedResultMeasureValue'] = \
                chem_aqui_ann_subset['ConvertedResultMeasureValue']/MCL_value
        
    elif chem_of_int == 'Nitrate':
        # remove other calculations of Nitrate
        # remove those row that do NOT have 'mg/l as N' as their units
        units_cond = chem_aqui_ann_subset['ResultMeasure/MeasureUnitCode'] != 'mg/l as N'
        chem_aqui_ann_subset = chem_aqui_ann_subset.loc[ units_cond, :]
        
        # Normalize the results to the MCL value
        chem_aqui_ann_subset['ConvertedResultMeasureValue'] = \
                chem_aqui_ann_subset['ResultMeasureValue']
        chem_aqui_ann_subset['NormalizedResultMeasureValue'] = \
                chem_aqui_ann_subset['ConvertedResultMeasureValue']/MCL_value
        
    # remove the 'nan' values in the final subsets
    final_subset = chem_aqui_ann_subset.loc[ ~pd.isnull(chem_aqui_ann_subset['NormalizedResultMeasureValue']), : ]
    
    # return the "final subset" of the data
    return final_subset
        
    
# =================== subsetting ^^^

The process and functions defined below eventually construct the figures.

The CA shapefile/outline used in each figure was obtained from the [California Open Data Portal](https://data.ca.gov/dataset/ca-geographic-boundaries/resource/e0107f72-1088-40a9-9443-6e657cadb805).

In [ ]:
# uploading the CA (California) border shapefile
border_dir = os.path.join( os.path.split(data_dir)[0], 'CA_State\\' )
ca_border  = gpd.read_file( (border_dir+'CA_State_TIGER2016.shp') )

In [ ]:
# =================== plotting vvvv
years_to_plot = ['2010',
                 '2011',
                 '2012',
                 '2013',
                 '2014',
                 '2015',
                 '2016',
                 '2017',
                 '2018']

Defining the function that plots each individual subplot.

In [ ]:
def plot_gw_data(data,
                 current_axis,
#                  rowid=0,
#                  colid=0,
                 chem_of_int='Arsenic',
                 year_of_int='2015'):
    
#     # setting up the subplot
#     current_axis = ax[rowid, colid]
#     current_axis = plt.axes(projection=ccrs.PlateCarree())

    # Setting the bounds/extent of the subplot
    current_axis.set_extent( [ca_max_lon,
                              ca_min_lon,
                              ca_min_lat,
                              ca_max_lat+0.25] )

    # add base layer image
    current_axis.add_image( Stamen('terrain-background'), 8 )

    # adding the CA shapefile
    current_axis.add_geometries(ca_border.geometry, 
                                crs=ccrs.Mercator(), 
                                edgecolor='black',
                                facecolor='none',
                                linewidth=1)

    # set the subplot title as the year
    current_axis.set_title( str(year_of_int), fontsize=16 )
    
    # label the y axis
    current_axis.text(-0.07, 0.55,
                      ('{}:' +'\n'+'[normalized to MCL]').format(chem_of_int),
                      va='bottom', ha='center',
                      rotation='vertical',rotation_mode='anchor',
                      transform=current_axis.transAxes,
                      fontsize=12)

    # selecting color map based on chemical of interest
    if chem_of_int == 'Arsenic':
        map_colors = 'Reds'
    else:
        map_colors = 'Blues'
        
    # add water quality data points
    current_axis.scatter(data['LongitudeMeasure'],
                         data['LatitudeMeasure'],
                         c=data['NormalizedResultMeasureValue'],
                         transform=ccrs.Geodetic(),
                         cmap=map_colors,
                         edgecolor='black',
                         vmin=0.0, vmax=1.0
                         )
    
    # convert the plot to a scalar mappable object to obtain a colorbar    
    sm = plt.cm.ScalarMappable(cmap=map_colors)
    sm._A = []
    sm.set_clim(vmin=0.0, vmax=1.0)
    cb = plt.colorbar(sm, fraction=0.043, pad=0.025, ax=current_axis)
    

The following `for loop` automates the subsetting of the data and the plotting of each subset of data into a unique subplot.

In [ ]:
# =================== Example
# fig, axs = plt.subplots(2,5, figsize=(15, 6), facecolor='w', edgecolor='k')
# fig.subplots_adjust(hspace = .5, wspace=.001)

# axs = axs.ravel()

# for i in range(10):

#     axs[i].contourf(np.random.rand(10,10),5,cmap=plt.cm.Oranges)
#     axs[i].set_title(str(250+i))

# =================== My stuff
# import geoplot
import geoplot as gplt
import geoplot.crs as gcrs
from cartopy.io.img_tiles import Stamen
import cartopy.crs as ccrs

# initializing the plot with subplots 
fig = plt.figure( figsize=(45, 10) )
fig.subplots_adjust( wspace=0.35, hspace=0.15)

#========= looping through the specified years

for iii in range( len(years_to_plot) ):
    # *** Plotting for Arsenic
    # plotting the As data on the top row
    subset_of_data = subset_data(mwgd, year_of_int=int(years_to_plot[iii]), chem_of_int='Arsenic' )

    # calling the function to plot - Arsenic
    test_ax = fig.add_subplot(2, 9, iii+1, projection=ccrs.PlateCarree()) 
    plot_gw_data(subset_of_data, test_ax, chem_of_int='Arsenic', year_of_int=int(years_to_plot[iii]))

    # *** Plotting for Nitrate
    # plotting the NO3 data on the bottom row
    subset_of_data = subset_data(mwgd, year_of_int=int(years_to_plot[iii]), chem_of_int='Nitrate' )

    # calling the function to plot - Nitrate (rowid=1)
    test_ax = fig.add_subplot(2, 9, iii+10, projection=ccrs.PlateCarree()) 
    plot_gw_data(subset_of_data, test_ax, chem_of_int='Nitrate', year_of_int=int(years_to_plot[iii]))

# saving the figure to the data directory
plt.savefig( os.path.join(data_dir, 'Plots\CA_TemporalDroughtPlot_CentralValleyAq_As_NO3.png'), 
             bbox_inches='tight')


C:\Users\allan\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\allan\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
